In [1]:
# train.py
import os
import random
import math
from pathlib import Path

import numpy as np
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import torchvision.transforms as T
import torchvision.models as models

from sklearn.model_selection import train_test_split

In [2]:
# Config
DATA_DIR = Path("/kaggle/input/csiro-biomass")  # Kaggle이면 이렇게, 로컬이면 바꿔줘
TRAIN_CSV = DATA_DIR / "train.csv"
IMAGE_ROOT = DATA_DIR  # image_path가 "train/xxx.jpg"라서 root는 DATA_DIR이면 됨

OUTPUT_DIR = Path("/kaggle/working/")
OUTPUT_DIR.mkdir(exist_ok=True, parents=True)

RANDOM_SEED = 42
BATCH_SIZE = 32
NUM_EPOCHS = 15
LR = 1e-3
IMAGE_SIZE = 224
NUM_WORKERS = 4
TARGET_NAMES = [
    "Dry_Clover_g",
    "Dry_Dead_g",
    "Dry_Green_g",
    "Dry_Total_g",
    "GDM_g",
]
N_TARGETS = len(TARGET_NAMES)

In [3]:
# set seed
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)


set_seed(RANDOM_SEED)

In [4]:
# data loading, pivot
def load_and_pivot_train(train_csv_path: Path):
    df = pd.read_csv(train_csv_path)


    df = df[["image_path", "target_name", "target"]]
    pivot = df.pivot_table(
        index="image_path",
        columns="target_name",
        values="target"
    )

    pivot = pivot.dropna(subset=TARGET_NAMES)
    pivot = pivot[TARGET_NAMES]
    pivot = pivot.reset_index()

    return pivot

In [5]:
# dataset
class BiomassDataset(Dataset):
    def __init__(self, df, image_root: Path, transforms=None, log_target=True):
        self.df = df.reset_index(drop=True)
        self.image_root = image_root
        self.transforms = transforms
        self.log_target = log_target

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        img_path = self.image_root / row["image_path"]
        image = Image.open(img_path).convert("RGB")

        if self.transforms:
            image = self.transforms(image)
        targets = row[TARGET_NAMES].values.astype("float32")

        if self.log_target:
            targets = np.log1p(targets)

        targets = torch.from_numpy(targets)

        return image, targets

In [6]:
# transforms
def get_transforms(image_size=224):
    train_tfms = T.Compose([
        T.Resize((image_size + 32, image_size + 32)),
        T.RandomResizedCrop(image_size, scale=(0.8, 1.0)),
        T.RandomHorizontalFlip(),
        T.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.05),
        T.ToTensor(),
        T.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225],
        ),
    ])

    val_tfms = T.Compose([
        T.Resize((image_size + 32, image_size + 32)),
        T.CenterCrop(image_size),
        T.ToTensor(),
        T.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225],
        ),
    ])

    return train_tfms, val_tfms

In [7]:
# model
class BiomassModel(nn.Module):
    def __init__(self, n_targets=5):
        super().__init__()
        try:
            backbone = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
        except Exception:
            backbone = models.resnet18(pretrained=False)

        in_features = backbone.fc.in_features
        backbone.fc = nn.Identity()

        self.backbone = backbone
        self.head = nn.Linear(in_features, n_targets)

    def forward(self, x):
        feat = self.backbone(x)
        out = self.head(feat)
        return out

In [8]:
# R^2 metric
def r2_score_torch(y_true, y_pred):
    y_true_mean = torch.mean(y_true, dim=0, keepdim=True)
    ss_tot = torch.sum((y_true - y_true_mean) ** 2, dim=0)
    ss_res = torch.sum((y_true - y_pred) ** 2, dim=0)
    eps = 1e-9
    r2_per_target = 1 - ss_res / (ss_tot + eps)
    r2_mean = torch.mean(r2_per_target).item()
    return r2_mean, r2_per_target.detach().cpu().numpy()

In [9]:
# train
def train_one_epoch(model, loader, optimizer, device, loss_fn):
    model.train()
    running_loss = 0.0

    for images, targets in loader:
        images = images.to(device)
        targets = targets.to(device)

        optimizer.zero_grad()
        preds = model(images)
        loss = loss_fn(preds, targets)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

    epoch_loss = running_loss / len(loader.dataset)
    return epoch_loss


@torch.no_grad()
def validate_one_epoch(model, loader, device, loss_fn):
    model.eval()
    running_loss = 0.0

    all_targets = []
    all_preds = []

    for images, targets in loader:
        images = images.to(device)
        targets = targets.to(device)

        preds = model(images)
        loss = loss_fn(preds, targets)

        running_loss += loss.item() * images.size(0)

        all_targets.append(targets.cpu())
        all_preds.append(preds.cpu())

    epoch_loss = running_loss / len(loader.dataset)

    all_targets = torch.cat(all_targets, dim=0)
    all_preds = torch.cat(all_preds, dim=0)

    r2_mean, r2_per_target = r2_score_torch(all_targets, all_preds)

    return epoch_loss, r2_mean, r2_per_target

In [10]:
# main
def main():
    set_seed(RANDOM_SEED)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Using device:", device)

    OUTPUT_DIR.mkdir(exist_ok=True, parents=True)

    pivot_df = load_and_pivot_train(TRAIN_CSV)
    print("Pivoted train size:", len(pivot_df))

    train_df, val_df = train_test_split(
        pivot_df,
        test_size=0.1,
        random_state=RANDOM_SEED,
        shuffle=True,
    )

    print("Train size:", len(train_df), "Val size:", len(val_df))

    train_tfms, val_tfms = get_transforms(IMAGE_SIZE)

    train_ds = BiomassDataset(train_df, IMAGE_ROOT, transforms=train_tfms, log_target=True)
    val_ds = BiomassDataset(val_df, IMAGE_ROOT, transforms=val_tfms, log_target=True)

    train_loader = DataLoader(
        train_ds,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=NUM_WORKERS,
        pin_memory=True,
    )

    val_loader = DataLoader(
        val_ds,
        batch_size=BATCH_SIZE,
        shuffle=False,
        num_workers=NUM_WORKERS,
        pin_memory=True,
    )

    model = BiomassModel(n_targets=N_TARGETS).to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=LR)
    loss_fn = nn.MSELoss()

    best_val_r2 = -1e9
    best_model_path = OUTPUT_DIR / "best_model.pth"

    history = []

    for epoch in range(1, NUM_EPOCHS + 1):
        print(f"\nEpoch {epoch}/{NUM_EPOCHS}")

        train_loss = train_one_epoch(model, train_loader, optimizer, device, loss_fn)
        print(f"  Train loss: {train_loss:.6f}")

        val_loss, val_w_r2, val_r2_per_target = validate_one_epoch(
            model, val_loader, device, loss_fn
        )
        print(f"  Val loss: {val_loss:.6f}")
        print(f"  Val CSIRO metric (weighted log-R2): {val_w_r2:.6f}")
        for name, r2v in zip(TARGET_NAMES, val_r2_per_target):
            print(f"    {name}: {r2v:.6f}")

        epoch_ckpt_path = OUTPUT_DIR / f"model_epoch_{epoch:03d}.pth"
        torch.save(
            {
                "epoch": epoch,
                "model_state_dict": model.state_dict(),
                "optimizer_state_dict": optimizer.state_dict(),
                "train_loss": train_loss,
                "val_loss": val_loss,
                "val_weighted_r2": val_w_r2,
                "val_r2_per_target": val_r2_per_target,
                "target_names": TARGET_NAMES,
            },
            epoch_ckpt_path,
        )
        print(f"  Saved epoch checkpoint to {epoch_ckpt_path}")

        if val_w_r2 > best_val_r2:
            best_val_r2 = val_w_r2
            torch.save(
                {
                    "epoch": epoch,
                    "model_state_dict": model.state_dict(),
                    "best_val_weighted_r2": best_val_r2,
                    "target_names": TARGET_NAMES,
                },
                best_model_path,
            )
            print(f"  >> New best model saved to {best_model_path} (score={best_val_r2:.6f})")

        row = {
            "epoch": epoch,
            "train_loss": float(train_loss),
            "val_loss": float(val_loss),
            "val_weighted_r2": float(val_w_r2),
        }
        for name, r2v in zip(TARGET_NAMES, val_r2_per_target):
            row[f"r2_{name}"] = float(r2v)
        history.append(row)

        log_df = pd.DataFrame(history)
        log_path = OUTPUT_DIR / "training_log.csv"
        log_df.to_csv(log_path, index=False)
        print(f"  Updated training log CSV at {log_path}")

    print("\nTraining finished.")
    print("Best val weighted R2:", best_val_r2)



if __name__ == "__main__":
    main()


Using device: cuda
Pivoted train size: 357
Train size: 321 Val size: 36


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)



Epoch 1/15
  Train loss: 3.420958
  Val loss: 195.636715
  Val CSIRO metric (weighted log-R2): -146.817978
    Dry_Clover_g: -7.519646
    Dry_Dead_g: -5.198360
    Dry_Green_g: -709.165955
    Dry_Total_g: -10.367054
    GDM_g: -1.838900
  Saved epoch checkpoint to /kaggle/working/model_epoch_001.pth
  >> New best model saved to /kaggle/working/best_model.pth (score=-146.817978)
  Updated training log CSV at /kaggle/working/training_log.csv

Epoch 2/15
  Train loss: 1.095264
  Val loss: 1.963795
  Val CSIRO metric (weighted log-R2): -1.565202
    Dry_Clover_g: -0.341017
    Dry_Dead_g: -2.182423
    Dry_Green_g: -0.224073
    Dry_Total_g: -4.211889
    GDM_g: -0.866610
  Saved epoch checkpoint to /kaggle/working/model_epoch_002.pth
  >> New best model saved to /kaggle/working/best_model.pth (score=-1.565202)
  Updated training log CSV at /kaggle/working/training_log.csv

Epoch 3/15
  Train loss: 0.985683
  Val loss: 1.232165
  Val CSIRO metric (weighted log-R2): -0.191833
    Dry_Clo

In [11]:
# make_submission.py
import os
from pathlib import Path

import numpy as np
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
import torchvision.models as models


DATA_DIR = Path("/kaggle/input/csiro-biomass")
TEST_CSV = DATA_DIR / "test.csv"
IMAGE_ROOT = DATA_DIR

MODEL_PATH = Path("/kaggle/working//best_model.pth")
SUBMISSION_PATH = Path("/kaggle/working/submission.csv")

In [12]:
# config
IMAGE_SIZE = 224
BATCH_SIZE = 64
NUM_WORKERS = 4

TARGET_NAMES = [
    "Dry_Clover_g",
    "Dry_Dead_g",
    "Dry_Green_g",
    "Dry_Total_g",
    "GDM_g",
]
N_TARGETS = len(TARGET_NAMES)

In [13]:
# model
class BiomassModel(nn.Module):
    def __init__(self, n_targets=5):
        super().__init__()
        try:
            backbone = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
        except Exception:
            backbone = models.resnet18(pretrained=False)

        in_features = backbone.fc.in_features
        backbone.fc = nn.Identity()

        self.backbone = backbone
        self.head = nn.Linear(in_features, n_targets)

    def forward(self, x):
        feat = self.backbone(x)
        out = self.head(feat)
        return out

def get_infer_transform(image_size=224):
    tfms = T.Compose([
        T.Resize((image_size + 32, image_size + 32)),
        T.CenterCrop(image_size),
        T.ToTensor(),
        T.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225],
        ),
    ])
    return tfms

In [14]:
# dataset
class TestImageDataset(Dataset):
    def __init__(self, image_paths, image_root: Path, transforms=None):
        self.image_paths = list(image_paths)
        self.image_root = image_root
        self.transforms = transforms

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_rel_path = self.image_paths[idx]
        img_path = self.image_root / img_rel_path

        image = Image.open(img_path).convert("RGB")

        if self.transforms:
            image = self.transforms(image)

        return image, img_rel_path

In [15]:
# main
def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Using device:", device)

    checkpoint = torch.load(MODEL_PATH, map_location=device)
    model = BiomassModel(n_targets=N_TARGETS)
    model.load_state_dict(checkpoint["model_state_dict"])
    model.to(device)
    model.eval()

    test_df = pd.read_csv(TEST_CSV)
    print("Test rows:", len(test_df))

    unique_image_paths = test_df["image_path"].unique()
    print("Unique test images:", len(unique_image_paths))

    tfms = get_infer_transform(IMAGE_SIZE)
    test_ds = TestImageDataset(unique_image_paths, IMAGE_ROOT, transforms=tfms)

    test_loader = DataLoader(
        test_ds,
        batch_size=BATCH_SIZE,
        shuffle=False,
        num_workers=NUM_WORKERS,
        pin_memory=True,
    )

    image_to_pred_log = {}

    with torch.no_grad():
        for images, img_rel_paths in test_loader:
            images = images.to(device)
            outputs = model(images)  

            outputs = outputs.cpu().numpy()
            for img_rel_path, pred_log in zip(img_rel_paths, outputs):
                image_to_pred_log[img_rel_path] = pred_log

    preds = []
    for _, row in test_df.iterrows():
        sample_id = row["sample_id"]
        img_rel_path = row["image_path"]
        target_name = row["target_name"]

        log_preds = image_to_pred_log[img_rel_path] 

        idx = TARGET_NAMES.index(target_name)

        val = np.expm1(log_preds[idx])

        val = float(max(val, 0.0))

        preds.append((sample_id, val))
    sub_df = pd.DataFrame(preds, columns=["sample_id", "target"])
    sub_df = sub_df.sort_values("sample_id") 
    sub_df.to_csv(SUBMISSION_PATH, index=False)

    print("Submission saved to:", SUBMISSION_PATH)


if __name__ == "__main__":
    main()

Using device: cuda


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


Test rows: 5
Unique test images: 1
Submission saved to: /kaggle/working/submission.csv
